In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Code based from:
- https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201808Results/Reg0828/2c-AddingControls-0828.ipynb

## Assign each reference text a unique identifier
- The "uuid" in the `otherreference.tsv` files is unique for every single reference
- Add unique identifier using haslib

In [ ]:
oc = pd.read_table("RawData/PatentsView/otherreference.tsv")
# Remove missing
oc = oc.dropna(how="any")

import hashlib
%time oc["ref_id"] = oc["text"].apply(lambda x: hashlib.sha1(str.encode(x)).hexdigest())

# Check it worked
print(len(oc.loc[oc["ref_id"].duplicated()]))

oc = oc[["ref_id", "patent_id"]]
oc["patent_id"] = oc["patent_id"].apply(lambda x: int(x) if x.isdigit() else np.nan)
oc = oc.loc[oc["patent_id"].notnull()]
oc["patent_id"] = oc["patent_id"].astype(int)

fastparquet.write("RawData/Cleaned/otherreference1016.parq", oc, compression="GZIP")

In [ ]:
# %time oc = fastparquet.ParquetFile("RawData/Cleaned/otherreference1016.parq").to_pandas()
%time oc = {n:g["ref_id"].tolist() for n,g in oc.groupby("patent_id")}

Add to all pair similarity files

In [ ]:
files = ["DataStore/2018-10/Reg0930/naics_name_all_1004.parq",
        "DataStore/2018-10/Reg0930/primclass_all_1004.parq",
        "DataStore/2018-06/newterm_pairs_sim_0620.parq",
        "DataStore/2018-08/newterms_pats_sim_cite_0903.parq",
        "DataStore/2018-07-P2/mutual_cited_0716.parq"]

for f in files:    
    rs = fastparquet.ParquetFile(f).to_pandas()
    
    if f == "DataStore/2018-06/newterm_pairs_sim_0620.parq":
        %time num_common_cites = (set(oc.get(tp, [])).intersection(set(oc.get(op, []))) for tp, op in zip(rs["lead_pat"], rs["follow_pat"]))
    else:
        %time num_common_cites = (set(oc.get(tp, [])).intersection(set(oc.get(op, []))) for tp, op in zip(rs["tp"], rs["op"]))
    
    %time rs["num_common_npc"] = [len(i) for i in num_common_cites]
    
    # Normalize
    if f in ["DataStore/2018-10/Reg0930/naics_name_all_1004.parq",
        "DataStore/2018-10/Reg0930/primclass_all_1004.parq"]:
        rs["norm_num_common_npc"] = np.nan
        rs.loc[rs["num_common_npc"].notnull(), "norm_num_common_npc"] = \
        scaler.fit_transform(rs.loc[rs["num_common_npc"].notnull(), "num_common_npc"].values.reshape(-1,1))
    fastparquet.write(f, rs, compression="GZIP")
    del(rs)

## Patent examiners

### Clean examiners data

In [9]:
# re = pd.read_table("RawData/PatentsView/rawexaminer.tsv")

re = re[["patent_id", "name_first", "name_last"]].dropna(how="any")

# Create id for each examiner
re["examiner"] = [f+" "+l for f,l in zip(re["name_first"], re["name_last"])]

import hashlib
%time re["examiner_id"] = re["examiner"].apply(lambda x: hashlib.sha1(str.encode(x)).hexdigest())

CPU times: user 16.1 s, sys: 796 ms, total: 16.9 s
Wall time: 17.5 s


In [10]:
re["patent_id"] = re["patent_id"].apply(lambda x: int(x) if x.isdigit() else np.nan)
re = re.loc[re["patent_id"].notnull()]
re["patent_id"] = re["patent_id"].astype(int)

In [11]:
print(len(re["examiner_id"].unique()))

79375


In [12]:
fastparquet.write("RawData/Cleaned/patexaminer1016.parq", re, compression="GZIP")

### Create dictionary of top names for fixed effects
- Examiners
- Lawyers
- MSAs
- Primary Classes

In [19]:
topn = {}
topn["examiner"] = re["examiner_id"].value_counts()[:100].index.tolist()

In [29]:
# Lawyers
ld = pd.read_csv("RawData/Cleaned/patent_lawyer.csv", index_col=0).drop_duplicates("patent")

topn["lawyer"] = ld["lawyer_id"].value_counts()[:100].index.tolist()

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [26]:
# MSAs & Primary classes
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "primclass", "inv_msa"])
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]


topn["inv_msa"] = pdf["inv_msa"].value_counts()[:100].index.tolist()
topn["primclass"] = pdf["primclass"].value_counts()[:100].index.tolist()
del(pdf)

In [31]:
# Dictionary for lawyer and examiners
re = dict(zip(re["patent_id"], re["examiner_id"]))
ld = dict(zip(ld["patent"], ld["lawyer_id"]))

In [42]:
mod_keys = ["naics_name", "primclass"]

for k in mod_keys:    
    rs = fastparquet.ParquetFile("DataStore/2018-10/Reg0930/{0}_all_1004.parq".format(k)).to_pandas()
    
    rs["tp_examiner"] = rs["tp"].map(re)
    rs["tp_lawyer"] = rs["tp"].map(ld)
    
    # Fixed effects
    for c in ["primclass", "inv_msa", "examiner", "lawyer"]:
        rs["tp_{0}_FE".format(c)] = rs["tp_{0}".format(c)].astype(str)
        rs.loc[~(rs["tp_{0}".format(c)].isin(topn[c])), "tp_{0}_FE".format(c)] = "other"
        
        print(rs["tp_{0}_FE".format(c)].value_counts()[:10])
        # Drop original
        rs = rs.drop("tp_{0}".format(c), 1)
        
    # New examiner match
    rs["examiner_match"] = (rs["tp"].map(re) == rs["op"].map(re))
    print("Examiner match")
    print(rs["examiner_match"].value_counts())
    
    print(rs.columns)
    
    # Common cited match
    rs["common_cited_match"] = (rs["num_common_cited"] >= 1)
    rs["common_npc_match"] = (rs["num_common_npc"] >= 1)
    print("Common cited")
    print(rs["common_cited_match"].value_counts())
    print("NPC")
    print(rs["common_npc_match"].value_counts())
    
    # Common establishment inventor a lot of missing values, replace with 0
    rs.loc[rs["common_est_inv"].isnull(), "common_est_inv"] = 0
    
    # Select columns
    rs = rs[['tp', 'op', 'sim_docvecs', 'sim_ldavecs', 'tp_gyear', 'tp_naics_name',
       'op_naics_name', 'op_primclass', 'op_inv_msa', 'inv_msa_match',
       'primclass_match', 'norm_sim_ldavecs', 'norm_sim_docvecs',
       'year_group', 'common_est_inv', 'common_pat_inv', 'lawyer_match',
       'num_common_cited', 'norm_num_common_cited', 'tp_pct_common_cited',
       'norm_tp_pct_common_cited', 'common_cited_match', "common_npc_match", 'mean_sim_docvecs_pc',
       'mean_sim_ldavecs_pc', 'mean_sim_docvecs_pc_msa',
       'mean_sim_ldavecs_pc_msa', 'norm_mean_sim_docvecs_pc',
       'norm_mean_sim_ldavecs_pc', 'norm_mean_sim_docvecs_pc_msa',
       'norm_mean_sim_ldavecs_pc_msa', 'sd_sim_docvecs_pc',
       'sd_sim_ldavecs_pc', 'sd_sim_docvecs_pc_msa', 'sd_sim_ldavecs_pc_msa',
       'pc_msa_greater_0', 'pc_msa_less_0', 'num_common_npc',
       'norm_num_common_npc', 'tp_primclass_FE', 'tp_inv_msa_FE',
       'tp_examiner_FE', 'tp_lawyer_FE', 'examiner_match']]

    # Save
    fastparquet.write("DataStore/2018-10/Reg1016/{0}_all_1016.parq".format(k), rs, compression="GZIP")
        
    
    

other    72337
514.0    33546
370.0    28712
435.0    27351
438.0    23681
424.0    22171
709.0    20187
455.0    20181
428.0    19745
600.0    18438
Name: tp_primclass_FE, dtype: int64
other                                                 146666
New York-Northern New Jersey-Long Island, NY-NJ-PA    111206
San Jose-Sunnyvale-Santa Clara, CA                    108874
Los Angeles-Long Beach-Santa Ana, CA                   85804
San Francisco-Oakland-Fremont, CA                      75895
Chicago-Joliet-Naperville, IL-IN-WI                    66269
Boston-Cambridge-Quincy, MA-NH                         64360
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD            41470
Detroit-Warren-Livonia, MI                             40644
San Diego-Carlsbad-San Marcos, CA                      37062
Name: tp_inv_msa_FE, dtype: int64
other                                       1402854
6de8dd1fabc379f2470e51f0d7613a78fb8add9e       1946
5dacd224066974a5b4abfe1c143f7544c6bb136c       1847
a09d2b4594182e

In [43]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1369833 entries, 0 to 1369832
Data columns (total 44 columns):
tp                              1369833 non-null int64
op                              1369833 non-null float64
sim_docvecs                     1354365 non-null float64
sim_ldavecs                     1355828 non-null float64
tp_gyear                        1369833 non-null int64
tp_naics_name                   1369833 non-null object
op_naics_name                   1369833 non-null object
op_primclass                    1369833 non-null float64
op_inv_msa                      1369833 non-null object
inv_msa_match                   1369833 non-null bool
primclass_match                 1369833 non-null bool
norm_sim_ldavecs                1355828 non-null float64
norm_sim_docvecs                1354365 non-null float64
year_group                      1369833 non-null object
common_est_inv                  1369833 non-null float32
common_pat_inv                  1369833 non-nu

### Add data for JTH Citations sample

In [3]:
rs = pd.read_pickle("DataStore/2018-07-P3/JTHReg0727/targ_cite_sim_reg_0727.pkl")
rs.columns

Index(['patent', 'tp_primclass', 'tp_gyear', 'perc_match_5', 'perc_match_10',
       'sim_ldavecs_cite_msa_match_5', 'sim_ldavecs_cite_msa_match_10',
       'sim_docvecs_cite_msa_match_5', 'sim_docvecs_cite_msa_match_10',
       'inv_msa_match', 'norm_perc_match_5', 'norm_perc_match_10',
       'norm_sim_ldavecs_cite_msa_match_5',
       'norm_sim_ldavecs_cite_msa_match_10',
       'norm_sim_docvecs_cite_msa_match_5',
       'norm_sim_docvecs_cite_msa_match_10', 'year_group'],
      dtype='object')

In [6]:
rs = pd.read_pickle("DataStore/2018-07-P3/JTHReg0727/targ_cite_sim_reg_0727.pkl")

# Patent examiner
re = fastparquet.ParquetFile("RawData/Cleaned/patexaminer1016.parq").to_pandas()

# Patent lawyers
ld = pd.read_csv("RawData/Cleaned/patent_lawyer.csv", index_col=0).drop_duplicates("patent")

# Top values
topn = {}
topn["examiner"] = re["examiner_id"].value_counts()[:100].index.tolist()
topn["lawyer"] = ld["lawyer_id"].value_counts()[:100].index.tolist()

# Use only what's in data
re = re.loc[re["patent_id"].isin(rs["patent"])]
ld = ld.loc[ld["patent"].isin(rs["patent"])]

re = dict(zip(re["patent_id"], re["examiner_id"]))
ld = dict(zip(ld["patent"], ld["lawyer_id"]))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
# Top MSAs & Primary classes
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "primclass", "inv_msa"])
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]

topn["inv_msa"] = pdf["inv_msa"].value_counts()[:100].index.tolist()
topn["primclass"] = pdf["primclass"].value_counts()[:100].index.tolist()

rs["tp_inv_msa"] = rs["patent"].map(dict(zip(pdf["patent"], pdf["inv_msa"])))
del(pdf)

In [9]:
rs["tp_examiner"] = rs["patent"].map(re)
rs["tp_lawyer"] = rs["patent"].map(ld)
    
# Fixed effects
for c in ["primclass", "inv_msa", "examiner", "lawyer"]:
    rs["tp_{0}_FE".format(c)] = rs["tp_{0}".format(c)].astype(str)
    rs.loc[~(rs["tp_{0}".format(c)].isin(topn[c])), "tp_{0}_FE".format(c)] = "other"

    print(rs["tp_{0}_FE".format(c)].value_counts()[:10])
    # Drop original
    try:
        rs = rs.drop("tp_{0}".format(c), 1)
    except Exception:
        pass

other    25316
370.0    11588
514.0    10524
435.0     9630
438.0     9452
709.0     9148
428.0     9014
424.0     8780
455.0     8670
600.0     8478
Name: tp_primclass_FE, dtype: int64
other                                                 97269
San Jose-Sunnyvale-Santa Clara, CA                    38161
New York-Northern New Jersey-Long Island, NY-NJ-PA    36591
Los Angeles-Long Beach-Santa Ana, CA                  29638
San Francisco-Oakland-Fremont, CA                     26763
Boston-Cambridge-Quincy, MA-NH                        22765
Chicago-Joliet-Naperville, IL-IN-WI                   22176
Minneapolis-St. Paul-Bloomington, MN-WI               13367
Detroit-Warren-Livonia, MI                            13071
Seattle-Tacoma-Bellevue, WA                           13000
Name: tp_inv_msa_FE, dtype: int64
other                                       526063
6de8dd1fabc379f2470e51f0d7613a78fb8add9e       958
c1b51fc89c11ff4c7ec7f64e20ba5582f07ba3be       841
5dacd224066974a5b4abfe1c143

In [12]:
rs["inv_msa_match"] = rs["inv_msa_match"].astype(bool)
rs.to_pickle("DataStore/2018-10/Reg1016/JTH_cite_reg_1026.pkl")

In [14]:
rs.sample(20)

,patent,tp_gyear,perc_match_5,perc_match_10,sim_ldavecs_cite_msa_match_5,sim_ldavecs_cite_msa_match_10,sim_docvecs_cite_msa_match_5,sim_docvecs_cite_msa_match_10,inv_msa_match,norm_perc_match_5,norm_perc_match_10,norm_sim_ldavecs_cite_msa_match_5,norm_sim_ldavecs_cite_msa_match_10,norm_sim_docvecs_cite_msa_match_5,norm_sim_docvecs_cite_msa_match_10,year_group,tp_primclass_FE,tp_inv_msa_FE,tp_examiner_FE,tp_lawyer_FE
157625,7885442,2011,0.000000,0.000000,NaN,NaN,NaN,NaN,True,-0.365524,-0.379346,NaN,NaN,NaN,NaN,2005-15,382.0,"Boston-Cambridge-Quincy, MA-NH",other,other
59616,6142300,2000,1.000000,0.250000,0.108738,0.108738,0.256381,0.256381,True,3.820074,0.775309,-1.436874,-1.436277,-0.036949,0.012436,1995-05,206.0,"New York-Northern New Jersey-Long Island, NY-N...",other,other
435809,5771274,1998,0.000000,0.136364,NaN,NaN,NaN,NaN,False,-0.365524,0.250466,NaN,NaN,NaN,NaN,1995-05,379.0,"Colorado Springs, CO",other,other
514364,7918805,2011,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,2005-15,600.0,"Worcester, MA",a460d161a22e9b38f18eb26afd4d3de2b24ecf1c,other
335029,4067406,1978,0.000000,0.000000,NaN,NaN,NaN,NaN,False,-0.365524,-0.379346,NaN,NaN,NaN,NaN,1975-85,175.0,"Los Angeles-Long Beach-Santa Ana, CA",other,other
75531,5222691,1993,0.000000,0.600000,NaN,0.671838,NaN,0.375728,True,-0.365524,2.391827,NaN,0.870487,NaN,0.777203,1985-95,244.0,"Bridgeport-Stamford-Norwalk, CT",other,other
66234,5469981,1995,NaN,0.000000,NaN,NaN,NaN,NaN,True,NaN,-0.379346,NaN,NaN,NaN,NaN,1995-05,216.0,"Poughkeepsie-Newburgh-Middletown, NY",other,other
436710,5644505,1997,0.000000,0.000000,NaN,NaN,NaN,NaN,False,-0.365524,-0.379346,NaN,NaN,NaN,NaN,1995-05,381.0,other,other,other
419836,7251172,2007,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,2005-15,365.0,"Colorado Springs, CO",other,other
480623,7318740,2008,0.000000,0.000000,NaN,NaN,NaN,NaN,False,-0.365524,-0.379346,NaN,NaN,NaN,NaN,2005-15,439.0,"Harrisburg-Carlisle, PA",other,other
